---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [126]:
import pandas as pd
import re
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)
text=pd.DataFrame({ 'line':df.values})
print(text.tail(20))


                                                  line
480  1. Esophageal cancer, dx: 2013, on FOLFOX with...
481                                      y1974 (all)\n
482  h/o restraining order by sister/mother in 1990...
483  sTexas Medical Center; Oklahoma for 2 weeks; 1...
484  Death of former partner in 2004 by overdose as...
485  Was "average" student.  "I didn't have too man...
486  Contemplating jumping off building - 1973 - di...
487  appendectomy s/p delivery 1992 Prior relevant ...
488  tProblems renal cell cancer : s/p nephrectomy ...
489      ran own business for 35 years, sold in 1985\n
490                                Lab: B12 969 2007\n
491                                 )and 8mo in 2009\n
492  .Moved to USA in 1986. Suffered from malnutrit...
493                                            r1978\n
494  . Went to Emerson, in Newfane Alaska. Started ...
495  1979 Family Psych History: Family History of S...
496  therapist and friend died in ~2006 Parental/Ca...
497       

In [127]:
def date_sorter():
    import pandas as pd
    import re
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    text=pd.DataFrame({ 'line':df.values})
    
    text['processed']=0

    def month_string_to_number(string):
        m = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr':4,
         'may':5,
         'jun':6,
         'jul':7,
         'aug':8,
         'sep':9,
         'oct':10,
         'nov':11,
         'dec':12
            }
        s = string.strip()[:3].lower()

        try:
            out = m[s]
            return out
        except:
            raise ValueError('Not a month')
        

    for i in range(len(text)):
        if (len(re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{4}',text.loc[i,'line']))>0) and (text.loc[i,'processed']==0):
            text.loc[i,'date']=re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{4}',text.loc[i,'line'])[0]        
            text.loc[i,'processed']=1
        
    for i in range(len(text)):        
        if (len(re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2}',text.loc[i,'line']))>0) and (text.loc[i,'processed']==0):
            pr=re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2}',text.loc[i,'line'])[0]
            text.loc[i,'processed']=2
            text.loc[i,'date']=pr[:-3].replace('-','/')+'/19'+pr[-2:]

    for i in range(len(text)):
        if ( (len(re.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* ?\d{2,4}',text.loc[i,'line']))>0) 
            and (text.loc[i,'processed']==0) ) :
            target=re.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* ?\d{2,4}',text.loc[i,'line'])[0]
            a=target.split()
            if a[0].isnumeric():
                text.loc[i,'date']=str(month_string_to_number(a[1]))+'/'+a[0]+'/'+a[2]
                text.loc[i,'processed']=3

        
    for i in range(len(text)):
        if ( (len(re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[. ] ?\d{2}[,. ] ?\d{2,4}',text.loc[i,'line']))>0) 
            and (text.loc[i,'processed']==0) ) :
            target=re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[. ] ?\d{2}[,. ] ?\d{2,4}',text.loc[i,'line'])[0]
            a=target.split()
            text.loc[i,'date']=str(month_string_to_number(a[0]))+'/'+a[1].replace(',','')+'/'+a[2]
            text.loc[i,'processed']=4


    for i in range(len(text)):
        if ( (len(re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[., ] ?\d{2,4}',text.loc[i,'line']))>0) 
            and (text.loc[i,'processed']==0) ) :
            target=re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[., ] ?\d{2,4}',text.loc[i,'line'])[0]
            a=target.split()
            text.loc[i,'date']=str(month_string_to_number(a[0]))+'/'+'1'+'/'+a[1]
            text.loc[i,'processed']=5       

    for i in range(len(text)):
        if (len(re.findall(r'\d{1,2}[/-]\d{4}',text.loc[i,'line']))>0) and (text.loc[i,'processed']==0) :
            target=re.findall(r'\d{1,2}[/-]\d{4}',text.loc[i,'line'])[0]
            a=target.split('/')
            text.loc[i,'date']=a[0]+'/'+'1'+'/'+a[1]
            text.loc[i,'processed']=6
    
    for i in range(len(text)):
        if (len(re.findall(r'\d{4}',text.loc[i,'line']))>0) and (text.loc[i,'processed']==0) :
            target=re.findall(r'\d{4}',text.loc[i,'line'])[0]
            text.loc[i,'date']='1'+'/'+'1'+'/'+target
            text.loc[i,'processed']=7
   
    text['date'] =  pd.to_datetime(text['date'], format='%m/%d/%Y')        
    text_new=text.sort_values('date')
    text_new['original']=text_new.index
    text_new.reset_index(inplace=True)
    text_new=text_new[['original']]

    
    return text_new.T.squeeze()
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: original, Length: 500, dtype: int64